In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt
from shapely.wkt import loads
import cv2 as cv
from nn_modules import Transformer_Decoder_Layer, Encoder

In [5]:
class Transformer_Decoder(nn.Module):
    def __init__(self,dim,num_heads=8,batch_size=3,n_classes = 2,num_patches=64):
        super().__init__()
        self.pos_emb = nn.Embedding(num_patches,dim)
        self.decoder_layer = Transformer_Decoder_Layer(dim,num_heads)
        self.q = nn.Embedding(n_classes,16)  
        self.batch_size = batch_size
        self.classifier = nn.Sequential(nn.Linear(n_classes,n_classes+1),nn.Softmax(-1))
        
    def forward(self,input):
        #for one layer
        features = input+self.pos_emb.weight
        q = self.q.weight.repeat(self.batch_size,1,1)
        q,attn = self.decoder_layer(q,features)        
        class_pred = self.classifier(attn.transpose(-1,-2))
        
        return(q,attn,class_pred)

In [6]:
decoder = Transformer_Decoder(16,8,3)
features = torch.rand(3,64,16)
q,attn,class_pred = decoder(features)